Let's look into the HSV color space
We'll start by creating our own HSV transformer

In [ ]:
# Starting with the usual imports
import numpy as np
import matplotlib as mp
from matplotlib import pyplot as plt
import cv2
from PIL import Image

In [ ]:
# Load the RIT image
baseImg = cv2.imread('RIT.jpg')
rgb = cv2.cvtColor(baseImg, cv2.COLOR_BGR2RGB)
plt.imshow(rgb, cmap = plt.cm.Spectral)
plt.show()

In [ ]:
# Let openCV convert for us
baseImg = cv2.imread('RIT.jpg')
img = cv2.cvtColor(baseImg, cv2.COLOR_BGR2HSV)
plt.imshow(img, cmap = plt.cm.Spectral)
plt.show()

In [ ]:
# Can we do this manually?
baseImg = cv2.imread('RIT.jpg')
# Split the image into Blue, Green, and Red channels
b, g, r = cv2.split(baseImg)

# Calculate element-wise maximum and minimum of r, g, and b
cmax = np.maximum(np.maximum(r, g), b)  # maximum of r, g, b
cmin = np.minimum(np.minimum(r, g), b)  # minimum of r, g, b

# Calculate the difference between cmax and cmin
diff = cmax - cmin

# Initialize the saturation array (s) with the same shape as cmax
s = np.zeros_like(cmax, dtype=np.float32)

# Create a mask for cmax == 0 (this will give s = 0)
mask = cmax == 0
# Set saturation to 0 where cmax is 0
s[mask] = 0

# For all other values of cmax (where cmax != 0), compute saturation
mask_non_zero = ~mask
s[mask_non_zero] = (diff[mask_non_zero] / cmax[mask_non_zero]) * 255

# Initialize the hue array (h) with the same shape as cmax
h = np.zeros_like(cmax, dtype=np.float32)

# Compute hue based on the maximum channel
# Check if cmax equals cmin (this would give h = 0)
mask_h = cmax == cmin
h[mask_h] = 0

# If cmax equals r, compute h
mask_h = (cmax == r) & (cmax != cmin)
h[mask_h] = (60 * ((g[mask_h] - b[mask_h]) / diff[mask_h]) + 360) % 360

# If cmax equals g, compute h
mask_h = (cmax == g) & (cmax != cmin)
h[mask_h] = (60 * ((b[mask_h] - r[mask_h]) / diff[mask_h]) + 120) % 360

# If cmax equals b, compute h
mask_h = (cmax == b) & (cmax != cmin)
h[mask_h] = (60 * ((r[mask_h] - g[mask_h]) / diff[mask_h]) + 240) % 360

# Ok, we took the very long way around. What does our manually 
# Convert the hue and saturation to an image
# Note: You need to convert the hue and saturation to proper color format to visualize them
hsv_image = np.stack([h, s, np.full_like(h, 255, dtype=np.float32)], axis=-1)
rgb_image = cv2.cvtColor(hsv_image.astype(np.uint8), cv2.COLOR_HSV2RGB)

# Plot the original and processed image
plt.figure(figsize=(12, 6))

# Original Image
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(baseImg)
plt.axis('off')

# Processed Image (in RGB)
plt.subplot(1, 2, 2)
plt.title('Processed Image (RGB to HSV)')
plt.imshow(rgb_image)
plt.axis('off')

plt.show()

Looks like we forgot to change our initial images r,g,b coordinate space!

This is why paying attention to pixel coordinates is so important.

This also shows that, by using transform equations, we can turn pixels into pretty much anything. The question is, "what changes are useful in CV?"
